# Weakly connected components

Weakly connected components was added in MADlib 1.12.  This notebook also includes helper functions based on the WCC output.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12-dev, git revision: rc/v1.9alpha-rc1-195-g85e89ef, cmake configuration time: Wed Jul 26 23:21:17 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create vertex and edge tables

In [103]:
%%sql 
DROP TABLE IF EXISTS vertex, edge;

CREATE TABLE vertex(
    id INTEGER
);

CREATE TABLE edge(
    src INTEGER,
    dest INTEGER,
    user_id INTEGER
);

INSERT INTO vertex VALUES
(0),
(1),
(2),
(3),
(4),
(5),
(6),
(10),
(11),
(12),
(13),
(14),
(15),
(16);

INSERT INTO edge VALUES
(0, 1, 1),
(0, 2, 1),
(1, 2, 1),
(1, 3, 1),
(2, 3, 1),
(2, 5, 1),
(2, 6, 1),
(3, 0, 1),
(5, 6, 1),
(6, 3, 1),
(10, 11, 2),
(10, 12, 2),
(11, 12, 2),
(11, 13, 2),
(12, 13, 2),
(13, 10, 2),
(15, 16, 2),
(15, 14, 2);

SELECT * FROM edge ORDER BY src, dest;

Done.
Done.
Done.
14 rows affected.
18 rows affected.
18 rows affected.


src,dest,user_id
0,1,1
0,2,1
1,2,1
1,3,1
2,3,1
2,5,1
2,6,1
3,0,1
5,6,1
6,3,1


# 2.  Find all weakly connected components

In [104]:
%%sql
DROP TABLE IF EXISTS wcc_out, wcc_out_summary;

SELECT madlib.weakly_connected_components(
                         'vertex',             -- Vertex table
                         'id',                 -- Vertix id column
                         'edge',               -- Edge table
                         'src=src, dest=dest', -- Comma delimted string of edge arguments
                         'wcc_out');           -- Output table of weakly connected components

SELECT * FROM wcc_out ORDER BY component_id, id;

Done.
1 rows affected.
14 rows affected.


id,component_id
0,0
1,0
2,0
3,0
5,0
6,0
4,4
10,10
11,10
12,10


# 3.  Weakly connected components with grouping

In [105]:
%%sql
DROP TABLE IF EXISTS wcc_out, wcc_out_summary;

SELECT madlib.weakly_connected_components(
                         'vertex',             -- Vertex table
                         'id',                 -- Vertix id column
                         'edge',               -- Edge table
                         'src=src, dest=dest', -- Comma delimted string of edge arguments
                         'wcc_out',            -- Output table of weakly connected components
                         'user_id');           -- Grouping column name

SELECT * FROM wcc_out ORDER BY component_id, id, user_id;

Done.
1 rows affected.
13 rows affected.


id,component_id,user_id
0,0,1
1,0,1
2,0,1
3,0,1
5,0,1
6,0,1
10,10,2
11,10,2
12,10,2
13,10,2


Note that vertex '4' is not identified as a separate component in the above result. This is because disconnected nodes cannot be assigned to a particular group with the current graph representation in MADlib.

# 4. Largest connected component

In [106]:
%%sql 
DROP TABLE IF EXISTS largest_cpt_table;

SELECT madlib.graph_wcc_largest_cpt(
                         'wcc_out',             -- WCC output table
                         'largest_cpt_table');  -- output table containing largest component ID

SELECT * FROM largest_cpt_table ORDER BY component_id;

Done.
1 rows affected.
2 rows affected.


user_id,component_id,num_vertices
1,0,6
2,10,4


# 5. Histogram of number of vertices per connected component

In [107]:
%%sql
DROP TABLE IF EXISTS histogram_table; 

SELECT madlib.graph_wcc_histogram(
                         'wcc_out',           -- WCC's output table
                         'histogram_table');  -- output table containing the histogram of vertices

SELECT * FROM histogram_table ORDER BY component_id;

Done.
1 rows affected.
3 rows affected.


user_id,component_id,num_vertices
1,0,6
2,10,4
2,14,3


# 6. Check if two vertices belong to the same component

In [108]:
%%sql
DROP TABLE IF EXISTS vc_table;

SELECT madlib.graph_wcc_vertex_check(
                         'wcc_out',    -- WCC output table
                         '14,15',      -- Pair of vertex IDs
                         'vc_table');  -- output table containing components that contain the two vertices

SELECT * FROM vc_table ORDER BY component_id;

Done.
1 rows affected.
1 rows affected.


user_id,component_id
2,14


# 7. Find all reachable vertices from a source vertex

In [109]:
%%sql
DROP TABLE IF EXISTS reach_table;

SELECT madlib.graph_wcc_reachable_vertices(
                         'wcc_out',         -- WCC output table
                         '0',               -- source vertex
                         'reach_table');    -- output table containing all vertices reachable from source vertex

SELECT * FROM reach_table ORDER BY component_id, dest;

Done.
1 rows affected.
5 rows affected.


user_id,component_id,dest
1,0,1
1,0,2
1,0,3
1,0,5
1,0,6


# 8. Count of connected components

In [110]:
%%sql
DROP TABLE IF EXISTS count_table;

SELECT madlib.graph_wcc_num_cpts(
                         'wcc_out',       -- WCC output table
                         'count_table');  -- output table containing number of components per group

SELECT * FROM count_table;

Done.
1 rows affected.
2 rows affected.


user_id,num_components
1,1
2,2
